#### Import Dependencies 

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from tqdm import tqdm
import copy
import pickle
from itertools import product
import pickle
from pytorchtools import EarlyStopping

In [2]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True 

#### Load Dataset
- Data must be in the following shape: 
    - X.shape = (number of trials for all subjects, timestamp, number of channels)
    - Y.shape = (number of trials for all subjects, 2) (shape of 2 for an array([subject id, label])

In [3]:
X, Y = pickle.load(open('CrossSubjects_MI.pkl', "rb"))

In [4]:
X.shape, Y.shape

((2328, 1750, 22), (2328, 2))

#### Define Variables

In [5]:
subject = 0 # subject to leave out from training and then test on 
total_subjects = 9 #total number of subjects in dataset
nchannels = 22 # number of EEG channels
batch_size = 12 # batch size for training 
batch_size1 = 1 # batch size for testing 
num_epoch = 1000 # number of epochs for training 
lr = 1e-4 #learning rate 
patience = 50 # patience for early stopping
timestamp = 1750 #timestamp recorded in trials
nclasses = 4 #number of label classes 
lrQM = 10 #learning rate multiplier
lmbd = 3  #lambda value 

#### Data Preprocessing

In [6]:
#Specify which subject you want to leave out 
def subject_out(X,Y,subject=0):
    XS = []
    YS = []
    XL = []
    YL = []
    for i in range(len(X)):
        if Y[i][0] == subject:
            XL.append(X[i])
            YL.append(Y[i])
        else:
            XS.append(X[i])
            YS.append(Y[i])
    XS = np.array(XS)
    YS = np.array(YS)
    XL = np.array(XL)
    YL = np.array(YL)
    return XS, YS, XL, YL

In [7]:
#Normalize the data per the number of EEG channels 
def prepsubdata(X,Y,nchannels):
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=0)
    
    scaler = StandardScaler()
    scaler.fit(X_train.reshape([-1,nchannels]))
    X1_train = scaler.transform(X_train.reshape([-1,nchannels])).reshape(X_train.shape)
    X1_test = scaler.transform(X_test.reshape([-1,nchannels])).reshape(X_test.shape)
    
    ########
    
    X1_train = torch.FloatTensor(X1_train).cuda()
    Y1_train = torch.FloatTensor(y_train).cuda()
    X1_test = torch.FloatTensor(X1_test).cuda()
    Y1_test = torch.FloatTensor(y_test).cuda()

    
    return X1_train, X1_test, Y1_train, Y1_test, scaler

def preploaders(X,Y,nchannels,batch_size):
    X_train, X_test, Y_train, Y_test, scaler = prepsubdata(X,Y,nchannels)
    train_data =[]
    for i in range(len(X_train)):
        train_data.append([X_train[i], Y_train[i]])
    test_data =[]
    for i in range(len(X_test)):
        test_data.append([X_test[i], Y_test[i]])
    
    trainloader = DataLoader(train_data, batch_size) 
    testloader = DataLoader(test_data, batch_size)    
    
    return trainloader, testloader, scaler


In [8]:
#Load the data for the test subject 
def prepdataloader(X,Y,scaler,nchannels,batch_size1):
    
    #Normalize
    XT = scaler.transform(X.reshape([-1,nchannels])).reshape(X.shape)
    
    XT = torch.FloatTensor(XT).cuda()
    Y = torch.FloatTensor(Y).cuda()
    

    
    #Prep Loader
    data =[]
    for i in range(len(XT)):
        data.append([XT[i], Y[i]])
    
    xloader = DataLoader(data,batch_size1)
    
    return xloader
 


#### Define Networks

In [9]:
class DTNet(nn.Module):
    def __init__(self):
        super(DTNet, self).__init__()
        self.T = timestamp
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, nchannels), padding = 0)

    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        return x


class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = timestamp
        
        #Continuation of Layer 1
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.dropout1 = nn.Dropout(p=0.25)
        self.elu = nn.ELU()
      
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer 
        self.fc1 = nn.Linear(4*2*109, nclasses) #4*2*109 - x.shape[1]*x.shape[2]*x.shape[3]

        

    def forward(self, x):
        
        #Continuation of Layer 1
        x = self.elu(x)
        x = self.batchnorm1(x)
        x = self.dropout1(x)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = self.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = self.dropout1(x)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = self.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = self.dropout1(x)
        x = self.pooling3(x)
   
        
        # FC Layer
        x = x.reshape([x.shape[0],x.shape[1]*x.shape[2]*x.shape[3]]) #flatten
        x = self.fc1(x) #removed sigmoid 
        return x 

#### Train & Test Model
- Run the following for cross-subject training/testing
- Can try out different hyperparameters

In [10]:
for batch_size, lr, lrQM, lmbd, subject in product([batch_size],[lr],[lrQM],[lmbd],range(total_subjects)): #iteratively selects subject to leave out
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    random.seed(0)

    XS, YS, XL, YL = subject_out(X,Y,subject)
    
    DT = DTNet().cuda()
    model = EEGNet().cuda()
    criterion = nn.CrossEntropyLoss()
    
    DTs = {}
    
    for i in range(total_subjects):
        DTs[i] = copy.deepcopy(DT)


    optimizer = optim.Adam(model.parameters() , lr=lr)
    
    params = []
    for m in DTs.values():
        params+=list(m.parameters())
    
    optimizerDT = optim.Adam(params, lr=lr*lrQM)

    
    trainingSUbjects = list(range(total_subjects))
    trainingSUbjects.remove(subject)
 

    L = lmbd
    trainLoss = [] 
    valLoss = []
    train_acc = []
    val_acc = []

    early_stopping = EarlyStopping(patience, verbose=True)

 

    trainloader, testloader, scaler = preploaders(XS, YS, nchannels, batch_size)
    


    for epoch in tqdm(range(num_epoch)):  
        training_loss = 0
        validation_loss = 0
        total_train = 0
        correct_train = 0

        for dt in DTs.values(): dt.train()
        model.train()

        for i, data in enumerate(trainloader,0):
            inputs, labels = data

        
            optimizer.zero_grad()
            optimizerDT.zero_grad()
            
            inputNew = [DTs[i](inp.unsqueeze(0).unsqueeze(0)) for inp, i in zip(inputs,labels[:,0].long().cpu().numpy())]
            inputNew = torch.cat(inputNew)
            
    
            outputs = model(inputNew)
            loss = criterion(outputs, labels[:,1].long())
            
            ws = [DTs[i].conv1.weight  for   i in  trainingSUbjects ]
            wavg = torch.mean(torch.stack(ws),axis=0)
            Rw = torch.mean(torch.stack([ torch.norm(DTs[i].conv1.weight-wavg)**2  for   i in  range(1,total_subjects) ]))

            bs = [DTs[i].conv1.bias  for   i in  trainingSUbjects ]
            bavg = torch.mean(torch.stack(bs),axis=0)
            Rb = torch.mean(torch.stack([ torch.norm(DTs[i].conv1.bias-bavg)**2  for   i in  range(1,total_subjects) ]))

            Rw, Rb
            loss = loss + L*(Rw+Rb)

            
            
            
            
            
            loss.backward()
            optimizer.step()
            optimizerDT.step()

            training_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)

            correct_train += (predicted == labels[:,1].long()).sum().item()



        correct = 0
        total = 0

        for dt in DTs.values(): dt.eval()
        model.eval() 

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                
                inputNew = [DTs[i](inp.unsqueeze(0).unsqueeze(0)) for inp, i in zip(inputs,labels[:,0].long().cpu().numpy())]
                inputNew = torch.cat(inputNew)
                

                outputs = model(inputNew)

                loss = criterion(outputs, labels[:,1].long()) 
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels[:,1].long()).sum().item()
                validation_loss += loss.item()



        acc_train = 100 * correct_train // total_train
        acc_val = 100 * correct // total
        training_loss = training_loss/(len(trainloader))
        validation_loss = validation_loss/(len(testloader))
        trainLoss.append(training_loss)
        valLoss.append(validation_loss)
        train_acc.append(acc_train)
        val_acc.append(acc_val)
        print(f"Training Loss: {training_loss}")
        print(f"Validation Loss: {validation_loss}")
        print(f'Training Accuracy of the network: {acc_train} %')
        print(f'Validation Accuracy of the network: {acc_val} %')


        early_stopping(validation_loss,model)
        if early_stopping.bestFound:
            iterationOfBestModel = epoch
            ws = [DTs[i].conv1.weight  for   i in  trainingSUbjects ]
            wavg = torch.mean(torch.stack(ws),axis=0)

            bs = [DTs[i].conv1.bias  for   i in  trainingSUbjects ]
            bavg = torch.mean(torch.stack(bs),axis=0)
            
        if early_stopping.early_stop:
            print("Early Stopping")
            break
    model = early_stopping.bestModel
    
    
    for dt in DTs.values(): dt.eval()
    model.eval()
    with torch.no_grad():
        DTs[subject].conv1.bias.set_(bavg)
        DTs[subject].conv1.weight.set_(wavg) 


    XL_train, XL_test, YL_train, YL_test = train_test_split(XL,YL, test_size=0.95, random_state=0)
    loader = prepdataloader(XL_train,YL_train,scaler,nchannels,batch_size1)
    subjectloader = prepdataloader(XL_test,YL_test,scaler,nchannels,batch_size1=XL_test.shape[0])
    
    optimizerDTNew = optim.Adam(DTs[subject].parameters(), lr=lr*lrQM)
    
    # Train additional epoch and optimize DTNew 
    EtrainLoss = [] 
    Etrain_acc = []
    Etest_acc = []
    
    model.eval()
    testLoss = []
    test_acc = []
    
    for epoch in tqdm(range(1)):
        etraining_loss = 0
        etotal_train = 0
        ecorrect_train = 0

        for dt in DTs.values(): dt.train()
        

        for i, data in enumerate(loader,0):
            inputs, labels = data


            optimizerDTNew.zero_grad()

            inputNew = [DTs[i](inp.unsqueeze(0).unsqueeze(0)) for inp, i in zip(inputs,labels[:,0].long().cpu().numpy())]
            inputNew = torch.cat(inputNew)


            outputs = model(inputNew)
            loss = criterion(outputs, labels[:,1].long())
            loss.backward()
            optimizerDTNew.step()



            etraining_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            etotal_train += labels.size(0)

            ecorrect_train += (predicted == labels[:,1].long()).sum().item()

        eacc_train = 100 * ecorrect_train // etotal_train
        etraining_loss = etraining_loss/(len(loader))
        EtrainLoss.append(etraining_loss)
        Etrain_acc.append(eacc_train)
        print(f"Extra Training Loss: {etraining_loss}")
        print(f'Extra Training Accuracy of the network: {eacc_train} %') 
        
        


        correct_test = 0
        total_test = 0
        testing_loss = 0
        

        for dt in DTs.values(): dt.eval()
     

        with torch.no_grad():
            for data in subjectloader:
                inputs, labels = data

                inputNew = [DTs[i](inp.unsqueeze(0).unsqueeze(0)) for inp, i in zip(inputs,labels[:,0].long().cpu().numpy())]
                inputNew = torch.cat(inputNew)


                outputs = model(inputNew)

                loss = criterion(outputs, labels[:,1].long()) 
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels[:,1].long()).sum().item()
                testing_loss += loss.item()

        acc_test = 100 * correct_test // total_test
        testing_loss = testing_loss/(len(subjectloader))
        testLoss.append(testing_loss)
        test_acc.append(acc_test)
        print(f"Extra Testing Loss: {testing_loss}")
        print(f'Extra Testing Accuracy of the network: {acc_test} %')

  0%|▏                                                                                                                                                                 | 1/1000 [00:06<1:51:45,  6.71s/it]

Training Loss: 1.5858209246266497
Validation Loss: 1.423403491292681
Training Accuracy of the network: 23 %
Validation Accuracy of the network: 27 %
Validation loss decreased (inf --> 1.423403).  Saving model ...


  0%|▎                                                                                                                                                                 | 2/1000 [00:11<1:29:50,  5.40s/it]

Training Loss: 1.5338701845085534
Validation Loss: 1.414586125101362
Training Accuracy of the network: 26 %
Validation Accuracy of the network: 27 %
Validation loss decreased (1.423403 --> 1.414586).  Saving model ...


  0%|▍                                                                                                                                                                 | 3/1000 [00:15<1:23:11,  5.01s/it]

...

Training Loss: 0.4176501717418432
Validation Loss: 1.1536770216056278
Training Accuracy of the network: 86 %
Validation Accuracy of the network: 52 %
EarlyStopping counter: 49 out of 50


 12%|███████████████████▋                                                                                                                                            | 123/1000 [09:38<1:08:42,  4.70s/it]


Training Loss: 0.3984500525785344
Validation Loss: 1.1432289166109904
Training Accuracy of the network: 86 %
Validation Accuracy of the network: 53 %
EarlyStopping counter: 50 out of 50
Early Stopping


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.91it/s]

Extra Training Loss: 0.9239721858704631
Extra Training Accuracy of the network: 72 %
Extra Testing Loss: 1.3857566118240356
Extra Testing Accuracy of the network: 52 %
